In [15]:
from collections import deque
import MCTS
import numpy as np
from ConnectX import ConnectX
from Agent import Agent_random, Agent_DNN
import tensorflow as tf


episodes = 400
outcomes = []
losses = []

agent = Agent_DNN()

import progressbar as pb
widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
timer = pb.ProgressBar(widgets=widget, maxval=episodes).start()

actor_optimizer = tf.keras.optimizers.Adam(1e-3)
critic_optimizer = tf.keras.optimizers.Adam(1e-3)

for e in range(episodes):

    mytree = MCTS.Node(ConnectX())
    vterm = []
    logterm = []
    with tf.GradientTape() as actor_tape, tf.GradientTape() as critic_tape:

        while mytree.winner is None:
            for _ in range(50):
                mytree.explore(agent)

            current_player = mytree.game.player
            mytree, (v, nn_v, p, nn_p) = mytree.next()
            mytree.detach_mother()

            # compute prob * log pi
            loglist = np.log(nn_p)*p

            # constant term to make sure if policy result = MCTS result, loss = 0
            constant = np.where(p>0, p*torch.log(p), 0.)
            logterm.append(-np.sum(loglist-constant))

            vterm.append(nn_v*current_player)


        # we compute the "policy_loss" for computing gradient
        outcome = mytree.winner
        outcomes.append(outcome)
        actor_loss = np.sum(np.stack(logterm))
        critic_loss = np.sum((np.stack(vterm)-outcome)**2)
        loss = actor_loss + critic_loss
        #loss = np.sum( (np.stack(vterm)-outcome)**2 + np.stack(logterm) )
        grad_actor = actor_tape.gradient(actor_loss, agent.actor.trainable_variables)
        grad_critic = critic_tape.gradient(critic_loss, agent.critic.trainable_variables)

        actor_optimizer.apply_gradients(zip(grad_actor, agent.actor.trainable_variables))
        critic_optimizer.apply_gradients(zip(grad_critic, agent.critic.trainable_variables))

        losses.append(float(loss))
    
    if (e+1)%50==0:
        print("game: ",e+1, ", mean loss: {:3.2f}".format(np.mean(losses[-20:])),
              ", recent outcomes: ", outcomes[-10:])
    del loss
    
    timer.update(e+1)
    
    
timer.finish()

ValueError: max() arg is an empty sequence

In [12]:
import MCTS

from copy import copy
import random
from ConnectX import ConnectX

from Agent import Agent_random, Agent_DNN
agent = Agent_DNN()

def Agent_Player_MCTS(game):
    mytree = MCTS.Node(copy(game))
    for _ in range(50):
        mytree.explore(agent)
   
    mytreenext, (v, nn_v, p, nn_p) = mytree.next(temperature=0.1)

In [13]:
game = ConnectX()
print(game)
Agent_Player_MCTS(game)

ValueError: ERROR: have no children